In [1]:
import numpy as np
import pandas as pd
from warnings import filterwarnings
from openai import OpenAI
filterwarnings(action='ignore')
from config import constants

class StatusAssigner:
    @classmethod
    def bad_status(cls, message):
        return {
            'status': 'error',
            'error_reason': message
    }
    @classmethod
    def good_status(cls):
        return {
            'status': 'accepted',
    }








class EMBEDDING(StatusAssigner):
    def __init__(self, api_key=constants.API_KEY):
        self.client = OpenAI(api_key=api_key)
        self.embedding_model = "text-embedding-3-small"
        self.df = None

    def _cosine_similarity(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def _get_embedding(self, text, model):
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def search_reviews(self, input_question, n=3, pprint=True):
        embedding = self._get_embedding(input_question, model='text-embedding-3-small')
        self.df['similarities'] = self.df['embedding'].apply(lambda x: self._cosine_similarity(x, embedding))

        res = self.good_status()
        res['data'] = self.df.sort_values('similarities', ascending=False).head(n)
        return res

    def load_embeddings(self, amount=10, most_similar_n=3, file_path='/home/np_1961/Downloads/BankFAQs.csv'):
        self.df = pd.read_csv(self.file_path).head(amount)
        self.df["embedding"] = self.df['Question'].apply(lambda x: self._get_embedding(x, model=self.embedding_model))
        self.df.to_csv('embeddings.csv')
        res = self.good_status()
        return res

    def ask_question(self, input_question):
        if self.df is None:
            return self.bad_status(message="first load the embeddings then call this request")

        res = self.good_status()
        res['answer'] = self.search_reviews(input_question)['Answer'].iloc[0]
        return res







class LLM(StatusAssigner):
    def __init__(self, api_key=constants.API_KEY):
        self.client=OpenAI(api_key=api_key)



    def send_to_model(self,question, file_contents='', prompt='', model="gpt-3.5-turbo"):
        completion = self.client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": file_contents},
                {"role": "user", "content": question}
            ]
        )

        return completion.choices[0].message.content.splitlines()





class RAG(StatusAssigner):
    def __init__(self, file_path):
        self.embedding=EMBEDDING(file_path=file_path)
        self.lmm=LLM()







ModuleNotFoundError: No module named 'openai'

In [3]:
res=pd.read_csv('res.csv')

In [4]:
res

,Unnamed: 0,Question,Answer,Class,embedding,similarities
0,0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security,"[0.009293698705732822, -0.03084607608616352, 0...",0.410012
1,1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security,"[-0.00970387365669012, -0.018280930817127228, ...",0.659425
2,2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security,"[-0.01753336936235428, -0.038931239396333694, ...",0.711325
3,3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security,"[-0.0282317902892828, 0.01619480364024639, 0.0...",0.729861
4,4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security,"[-0.025099100545048714, -0.007126737851649523,...",0.867408
5,5,Can I use the same IVR Password to perform mu...,"No, each IVR password can be used only for a m...",security,"[0.0007861416088417172, -0.03542797267436981, ...",0.697354
6,6,Can I generate multiple IVR Passwords,"No, only one IVR password can be generated at ...",security,"[0.004773120861500502, -0.030521269887685776, ...",0.718223
7,7,How do I register for IVR password,There is no registration process. However you ...,security,"[-0.017454277724027634, -0.004589803051203489,...",0.807110
8,8,What is IVR Password and how does it work,To make telephonic (IVR) transactions more sec...,security,"[-0.026408161967992783, -0.007872556336224079,...",0.810158
9,9,In how much time will the IVR Password be deli...,IVR 3D Secure password is delivered instantly....,security,"[0.02888462133705616, 0.015391342341899872, 0....",0.647869


In [5]:

res.sort_values(by='similarities').iloc[-1]

Unnamed: 0                                                      4
Question                        How can I obtain an IVR Password 
Answer          By Sending SMS request: Send an SMS 'PWD<space...
Class                                                    security
embedding       [-0.025099100545048714, -0.007126737851649523,...
similarities                                             0.867408
Name: 4, dtype: object

In [2]:
!pip install flask

  Using cached itsdangerous-2.1.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 389.9 kB/s eta 0:00:00 kB/s eta 0:00:01
Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 1.1 MB/s eta 0:00:00m eta 0:00:010:01:01


In [3]:

from flask import Flask, jsonify, request
from helpers import RAG

chatbot = RAG()
app = Flask(__name__)

@app.route('/load_model', methods=['POST'])
def load_model():
    """
    No input
    """

    res = chatbot.embedding.load_embeddings(amount=3)
    return jsonify(res)

@app.route('/search', methods=['POST'])
def search():
    """
    input_question - str
    """

    if "input_question" not in request.form:
        return jsonify({
            "status": 500, "message": "input_question not defined"}), 500
    input_question = request.form['input_question']
    res = chatbot.embedding.ask_question(input_question=input_question)
    return jsonify(res)


@app.route('/search_and_generate', methods=['POST'])
def llm_generate():
    """
    input_question - str
    """

    if "input_question" not in request.files:
        return jsonify({
            "status": 500, "message": "input_question not defined"}), 500
    input_question = request.files['input_question'].readlines()

    res = chatbot.embedding.ask_question(input_question=input_question)
    return jsonify(res)

    #TODO
    # add llm model gpt3.5

    return "<h1>Not implemented</h1>"


@app.route('/document_search', methods=['POST'])
def document_search():
    """
    input_question - str
    """

    # request.files
    return "<h1>Not implemented</h1>"


@app.route('/', methods=['GET'])
def start():
    return "<h1>Hello this is FAQ api</h1>"


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/np_1961/anaconda3/envs/ai/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/np_1961/anaconda3/envs/ai/lib/python3.11/site-packages/traitlets/config/application.py", line 991, in launch_instance
    app.initialize(argv)
  File "/home/np_1961/anaconda3/envs/ai/lib/python3.11/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
          

SystemExit: 1